In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

from fastai import *
from fastai.vision import *
from ipyexperiments import *
from fastai.basic_train import *
import torchvision

Maybe use `loss(reduction="sum")` and average before `real_step()`

### MNIST batch size = 32, no accumulation

`effective batch size = 32`

In [3]:
import sys
sys.path.append("../dev")
from data_utils import seed_everything
from fastai.train import *

In [4]:
# seed everything for reproducibility
seed_everything(42)

In [5]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path, bs=32)

In [6]:
learn = create_cnn(data, torchvision.models.vgg11, metrics=accuracy)

In [7]:
learn = create_cnn(data, torchvision.models.vgg11, metrics=accuracy,
                   custom_head=nn.Sequential(*([AdaptiveConcatPool2d(), Flatten()]
                                             + bn_drop_lin(1024, 512, bn=False, p=0.5, actn=nn.ReLU())
                                             + bn_drop_lin(512, 2, bn=False, p=0.5))))

In [9]:
learn.loss_func = CrossEntropyFlat(reduction='mean')

In [10]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.125997,0.059610,0.975957


### MNIST effective batch size=32, accumulate

`effective batch size = 32 (bs x n_step)`

In [56]:
# seed everything for reproducibility
seed_everything(42)

In [57]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path, bs=1)

In [58]:
learn = create_cnn(data, torchvision.models.vgg11, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=32)],
                   custom_head=nn.Sequential(*([AdaptiveConcatPool2d(), Flatten()]
                                             + bn_drop_lin(1024, 512, bn=False, p=0.5, actn=nn.ReLU())
                                             + bn_drop_lin(512, 2, bn=False, p=0.5))))

In [59]:
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [60]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.324528,0.058682,0.977920


In [61]:
# seed everything for reproducibility
seed_everything(42)

In [62]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path, bs=2)

In [63]:
learn = create_cnn(data, torchvision.models.vgg11, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=16)],
                   custom_head=nn.Sequential(*([AdaptiveConcatPool2d(), Flatten()]
                                             + bn_drop_lin(1024, 512, bn=False, p=0.5, actn=nn.ReLU())
                                             + bn_drop_lin(512, 2, bn=False, p=0.5))))

In [64]:
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [65]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.380728,0.148829,0.974975


In [66]:
# seed everything for reproducibility
seed_everything(42)

In [67]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path, bs=4)

In [68]:
learn = create_cnn(data, torchvision.models.vgg11, metrics=accuracy,
                   callback_fns=[partial(AccumulateStepper, n_step=8)],
                   custom_head=nn.Sequential(*([AdaptiveConcatPool2d(), Flatten()]
                                             + bn_drop_lin(1024, 512, bn=False, p=0.5, actn=nn.ReLU())
                                             + bn_drop_lin(512, 2, bn=False, p=0.5))))

In [69]:
learn.loss_func = CrossEntropyFlat(reduction='sum')

In [70]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy
1,0.389837,0.263747,0.975957
